# **Network Data Source**: Host <-> Network Join
------------------

In [1]:
import pandas as pd, numpy as np, networkx as nx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2
from sklearn.linear_model.logistic import LogisticRegression
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = 500

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df_zeek = pd.read_json("df_zeek_labelled3.json", lines = True)
df_zeek.replace('nan', np.nan, inplace=True)
print(type(df_zeek))
print(df_zeek.shape)
print(df_zeek.EventID.value_counts())
df_zeek.head()

<class 'pandas.core.frame.DataFrame'>
(2248, 158)
5156.0    67
3.0       67
Name: EventID, dtype: int64


,@stream,@system,@proc,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_l2_addr,resp_l2_addr,rtt,named_pipe,endpoint,operation,trans_id,query,qclass,qclass_name,qtype,qtype_name,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected,analyzer,failure_reason,fuid,tx_hosts,rx_hosts,conn_uids,source,depth,analyzers,mime_type,filename,is_orig,seen_bytes,total_bytes,missing_bytes,overflow_bytes,timedout,md5,sha1,request_type,client,success,error_msg,till,forwardable,renewable,cipher,note,msg,sub,src,dst,p,actions,suppress_for,action,path,name,size,times_modified,times_accessed,times_created,times_changed,share_type,version,curve,server_name,resumed,established,cert_chain_fuids,client_cert_chain_fuids,subject,issuer,validation_status,resp_certificate_sha1,ja3,ja3s,notice,peer,addl,id,certificate_version,certificate_serial,certificate_subject,certificate_issuer,certificate_not_valid_before,certificate_not_valid_after,certificate_key_alg,certificate_sig_alg,certificate_key_type,certificate_key_length,certificate_exponent,basic_constraints_ca,extracted,extracted_cutoff,trans_depth,method,host,uri,user_agent,request_body_len,response_body_len,status_code,status_msg,tags,resp_fuids,resp_mime_types,client_header_names,orig_fuids,orig_mime_types,machine,compile_ts,os,subsystem,is_exe,is_64bit,uses_aslr,uses_dep,uses_code_integrity,uses_seh,has_import_table,has_export_table,has_cert_table,has_debug_data,section_names,Source_IP,Source_Port_x,Destination_IP,Destination_Port,ts_pd,EventID,TP_Binary,TP_Label,@timestamp,Source_Port_y
0,conn,bobs.bigwheel.local,zeek,1.588205e+09,Cvf4XX17hSAgXDdGEd,10.0.1.6,54243.0,10.0.0.4,53.0,udp,dns,0.001528,39.0,55.0,SF,0.0,Dd,1.0,67.0,1.0,83.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,54243.0,10.0.0.4,53.0,2020-04-30 00:06:39.824369907+00:00,NaN,False,0,NaN,NaN
1,conn,bobs.bigwheel.local,zeek,1.588205e+09,CJ21Le4zsTUcyKKi98,10.0.1.6,56880.0,10.0.0.4,445.0,tcp,"gssapi,smb,krb",10.761077,4020.0,1681.0,RSTO,0.0,ShADdaR,12.0,1307.0,11.0,2133.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,56880.0,10.0.0.4,445.0,2020-04-30 00:06:39.827696085+00:00,NaN,False,0,NaN,NaN
2,conn,bobs.bigwheel.local,zeek,1.588205e+09,CnOP7t1eGGHf6LFfuk,10.0.1.6,65108.0,10.0.0.4,53.0,udp,dns,0.001599,36.0,104.0,SF,0.0,Dd,1.0,64.0,1.0,132.0,00:0d:3a:13:e3:69,12:34:56:78:9a:bc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0.1.6,65108.0,10.0.0.4,53.0,2020-04-30 00:07:59.953876019+00:00,

In [3]:
df_label = pd.read_json('./df_labelled_coalesce.json', lines=True)
df_label['TP_Binary'] = df_label.TP_Label!='0'
df_label.replace('-', np.nan, inplace=True)
print(type(df_label))
print(df_label.shape)
print(df_label.EventID.value_counts())
df_label.head(1)

<class 'pandas.core.frame.DataFrame'>
(198760, 390)
12       61158
10       39286
7        20259
13       17542
4658     10973
         ...  
172          1
46           1
50103        1
306          1
41           1
Name: EventID, Length: 165, dtype: int64


,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host,port,tags,TP_Label,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,SubLayerName,SubLayerType,Default SD String:,ClassId,ClassName,CompatibleIds,DeviceDescription,DeviceId,LocationInformation,VendorIds,TargetProcessName,SecurityPackageName,AdditionalInfo,AdditionalInfo2,ProviderType,RuleAttr,AdapterName,AdapterSuffixName,DnsServerList,ErrorCode,Ipaddress,Sent UpdateServer,TaskContentNew,TaskName,ActiveProfile,MiniportName,MiniportNameLen,GroupPolicyApplied,LogDroppedPacketsEnabled,LogSuccessfulConnectionsEnabled,MulticastFlowsEnabled,OperationMode,Profile,RemoteAdminEnabled,TargetInfo,TargetLogonGuid,TargetServerName,ClientCreationTime,TimeSource,ImagePath,StartType,ProfileChanged,BootType,LoadOptions,EntryCount,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,

In [4]:
df_label.TP_Binary.value_counts()

False    195322
True       3438
Name: TP_Binary, dtype: int64

In [5]:
# Propagate network data label to all connections within a uid?

In [6]:
# Looking for False Positives with TP_Binary=True and ts outside of true positive window.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_zeek.groupby(['ts', 'EventID', 'TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'])['@timestamp', 'EventID', 'TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


@timestamp  \
ts           EventID TP_Binary uid                Source_IP Destination_IP Destination_Port               
1.588205e+09 3.0     True      CUFN2104UUWgAE48   10.0.1.4  192.168.0.5    1234.0                     1   
             5156.0  True      CUFN2104UUWgAE48   10.0.1.4  192.168.0.5    1234.0                     1   
1.588206e+09 3.0     True      CnfxOnz9EEuMywvih  10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CnfxOnz9EEuMywvih  10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CRxmG32y1yTI2Odtt9 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CRxmG32y1yTI2Odtt9 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      C4r8wf2BmPKRfanAF6 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      C4r8wf2BmPKRfanAF6 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CFmtFn4oHdLD2jVYD2 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CFmtFn4oHdLD2jVYD2 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CN30222JydecBjLXGh 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             5156.0  True      CzUOBX1HtxiLDcLRXg 10.0.1.4  192.168.0.5    443.0                      2   
             3.0     True      C6aOhX3MGEBfgSoXW6 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      C6aOhX3MGEBfgSoXW6 10.0.1.4  10.0.0.4       389.0                      2   
1.588207e+09 3.0     True      CzJQsD4vDzP1O4LDK1 10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      CzJQsD4vDzP1O4LDK1 10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      C0vu7WGlpKLjL8Qhc  10.0.1.4  10.0.0.4       389.0                      2   
             5156.0  True      C0vu7WGlpKLjL8Qhc  10.0.1.4  10.0.0.4       389.0                      2   
             3.0     True      CgFqEH2AArPQ4DLOYf 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CgFqEH2AArPQ4DLOYf 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CGNgVO1OlaEMzjq5A3 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CGNgVO1OlaEMzjq5A3 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CjqEFp2wB12h7DEGt2 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CjqEFp2wB12h7DEGt2 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CWHEjpiltZznVrMaa  10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CWHEjpiltZznVrMaa  10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True      CRTaeg2G66cIB9TRm6 10.0.1.4  10.0.1.6       5985.0                     4   
             5156.0  True      CRTaeg2G66cIB9TRm6 10.0.1.4  10.0.1.6       5985.0                     4   
             3.0     True  

In [7]:
# Group by ts vs TP_Binary shows if label accurately captures the scope of the connection. 
# No need to propagate label using uid. 1 to 1.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_zeek.groupby(['uid', 'TP_Binary'])['@timestamp', 'EventID', 'TP_Binary', 'uid', 'Source_IP', 'Destination_IP', 'Destination_Port'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,,@timestamp,EventID,TP_Binary,uid,Source_IP,Destination_IP,Destination_Port
uid,TP_Binary,,,,,,,
C03eJ14HreyLlDSYXb,False,0,0,1,1,1,1,1
C0Ah071D3JnoodwJO2,False,0,0,2,2,2,2,2
C0LmMn33GTjEqE0YKc,False,0,0,3,3,3,3,3
C0SIhd2lBq9aEbOI1d,False,0,0,2,2,2,2,2
C0XYv11LLkSOaKmcOa,True,8,8,8,8,8,8,8
C0ch1M1Oka8xzF0cR4,False,0,0,2,2,2,2,2
C0texO2ImscKqridG4,False,0,0,2,2,2,2,2
C0vu7WGlpKLjL8Qhc,True,4,4,4,4,4,4,4
C1LaI03eBfyLTLual1,False,0,0,1,1,1,1,1


In [8]:
# Left join host<->network on @timestamp, EventID

In [9]:
df_zeek.TP_Binary.value_counts()

False    2114
True      134
Name: TP_Binary, dtype: int64

In [10]:
# I can test the certainty of the TP_Binary label on the JOIN. 
# See the diff between TP_Binary_x and TP_Binary_y! ID False Positives.
# See if the host->network label prop based on ['Destination_IP', 'Source_IP', 'Destination_Port'] is legit!

In [11]:
df_label.shape

(198760, 390)

In [12]:
df_zeek.shape

(2248, 158)

In [13]:
# Join Host<->Network based on ['EventID', '@timestamp']
# Worked! Trying outer
#df_label = pd.merge(
#    left = df_label,
#    right = df_zeek,
#    how = 'left',
#    on = ['EventID', '@timestamp']
#)

In [14]:
#df_label.shape

In [15]:
# Join Host<->Network based on 
df_label = pd.merge(
    left = df_label,
    right = df_zeek,
    how = 'outer',
    on = ['@timestamp', 'EventID']
)

In [16]:
df_label.shape

(201048, 546)

In [17]:
df_label.head(1)

,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host_x,port,tags_x,TP_Label_x,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,ProcessGuid,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,...,KeyFilePath,ProviderContextKey,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Group,IdleImplementation,IdleStateCount,MaximumPerformancePercent,MinimumPerformancePercent,MinimumThrottlePercent,NominalFreq

In [18]:
df_label.TP_Binary_x.value_counts()

False    195354
True       3580
Name: TP_Binary_x, dtype: int64

In [19]:
df_label.TP_Binary_y.value_counts()

False    2114
True      196
Name: TP_Binary_y, dtype: int64

In [20]:
# Looking for differences between TP_Label in the Host Data vs the Network Data.
#     -> So 12 hits on Host Data labelled benign that was labelled malicious in the Network Data.
#     -> Could still be accurate label, revealing a shortcoming in the original host SQL labelling technique.
#     -> the 24 hits on TP_Label_x == NaN and TP_Label_y == 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.groupby(['TP_Label_x', 'TP_Binary_x', 'TP_Label_y', 'TP_Binary_y'])['@timestamp', 'EventID'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


@timestamp  \
TP_Label_x                        TP_Binary_x TP_Label_y                        TP_Binary_y               
0                                 False       0, 8.A.1. Remote System Discovery True                 12   
                                              0, 8.A.2. Remote System Discovery True                 24   
0, 1.A.3. Uncommonly Used Port    True        0, 1.A.3. Uncommonly Used Port    True                  2   
0, 3.B.3. Commonly Used Port      True        0, 3.B.3. Commonly Used Port      True                 30   
0, 8.A.1. Remote System Discovery True        0, 8.A.1. Remote System Discovery True                 48   
0, 8.A.2. Remote System Discovery True        0, 8.A.2. Remote System Discovery True                 80   

                                                                                             EventID  
TP_Label_x                        TP_Binary_x TP_Label_y                        TP_Binary_y           
0                                 False       0, 8.A.1. Remote System Discovery True              12  
                                              0, 8.A.2. Remote System Discovery True              24  
0, 1.A.3. Uncommonly Used Port    True        0, 1.A.3. Uncommonly Used Port    True               2  
0, 3.B.3. Commonly Used Port      True        0, 3.B.3. Commonly Used Port      True              30  
0, 8.A.1. Remote System Discovery True        0, 8.A.1. Remote System Discovery True              48  
0, 8.A.2. Remote System Discovery True        0, 8.A.2. Remote System Discovery True              80

In [21]:
# Dive into the potential false positives.
#     -> Looks like the network label picked up the:
#           -> 10.0.1.4 -> 10.0.0.4 Port 88 kerberos auth attempts (20x)  -> could go either way tp/fp
#           -> 10.0.1.4 -> 10.0.1.6 Port 5985 WinRM(PowerShell Remoting) (8x) -> maps to emulation plan. workstation->workstation WinRM is generally tp malicious
#           -> 10.0.1.4 -> 10.0.0.4 Port 53 DNS (8x) -> likely FP. DNS between a domain workstation and it's domain controller is normal.
#     -> I'll just treat TP_Label_x or TP_Binary_x as the correct label going forward.
df_label.loc[(df_label.TP_Label_x == '0') & (df_label.TP_Binary_y == 'True')].groupby(['Message'])['EventID', 'Message'].count()

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


,EventID,Message
Message,,
Network connection detected:\r\nRuleName: -\r\nUtcTime: 2020-05-02 03:09:04.485\r\nProcessGuid: {47ab858c-cad9-5eac-0c00-000000000400}\r\nProcessId: 736\r\nImage: C:\Windows\System32\lsass.exe\r\nUser: NT AUTHORITY\SYSTEM\r\nProtocol: tcp\r\nInitiated: true\r\nSourceIsIpv6: false\r\nSourceIp: 10.0.1.4\r\nSourceHostname: -\r\nSourcePort: 59956\r\nSourcePortName: -\r\nDestinationIsIpv6: false\r\nDestinationIp: 10.0.0.4\r\nDestinationHostname: -\r\nDestinationPort: 88\r\nDestinationPortName: -,12,12
Network connection detected:\r\nRuleName: -\r\nUtcTime: 2020-05-02 03:09:23.794\r\nProcessGuid: {47ab858c-cad9-5eac-0c00-000000000400}\r\nProcessId: 736\r\nImage: C:\Windows\System32\lsass.exe\r\nUser: NT AUTHORITY\SYSTEM\r\nProtocol: tcp\r\nInitiated: true\r\nSourceIsIpv6: false\r\nSourceIp: 10.0.1.4\r\nSourceHostname: -\r\nSourcePort: 59958\r\nSourcePortName: -\r\nDestinationIsIpv6: false\r\nDestinationIp: 10.0.0.4\r\nDestinationHostname: -\r\nDestinationPort: 88\r\nDestinationPortName: -,8,8
Network connection detected:\r\nRuleName: -\r\nUtcTime: 2020-05-02 03:09:29.713\r\nProcessGuid: {5aa8ec29-cad1-5eac-0100-000000000400}\r\nProcessId: 4\r\nImage: System\r\nUser: NT AUTHORITY\SYSTEM\r\nProtocol: tcp\r\nInitiated: false\r\nSourceIsIpv6: false\r\nSourceIp: 10.0.1.4\r\nSourceHostname: -\r\nSourcePort: 59962\r\nSourcePortName: -\r\nDestinationIsIpv6: false\r\nDestinationIp: 10.0.1.6\r\nDestinationHostname: -\r\nDestinationPort: 5985\r\nDestinationPortName: -,8,8
The Windows Filtering Platform has permitted a connection.\r\n\r\nApplication Information:\r\n\tProcess ID:\t\t1636\r\n\tApplication Name:\t\device\harddiskvolume2\windows\system32\svchost.exe\r\n\r\nNetwork Information:\r\n\tDirection:\t\tOutbound\r\n\tSource Address:\t\t10.0.1.4\r\n\tSource Port:\t\t59445\r\n\tDestination Address:\t10.0.0.4\r\n\tDestination Port:\t\t53\r\n\tProtocol:\t\t17\r\n\r\nFilter Information:\r\n\tFilter Run-Time ID:\t68659\r\n\tLayer Name:\t\tConnect\r\n\tLayer Run-Time ID:\t48,8,8


In [22]:
# Write out host<->network outer join to disk
# Spark method crashed
# Trying pandas to_json
#df_label.to_json('./df_label_zeek_outer1.json')
# 1.7 G on disk...

In [23]:
# Kernel Crash on attempted read. Testing in new notebook with fresh system reboot.
#df_label = pd.read_json('./df_label_zeek_outer1.json')

In [ ]:
# Time Window Label Attempt1
# df_zeek['ts_pd'] = df_zeek["ts"].apply(lambda x: (pd.to_datetime(x, unit='s', utc=True)))
# df_label['ts_pd'] = df_label['@timestamp'].apply(lambda x: (pd.Timestamp(x, unit='s')))

In [24]:
df_label['time_stamp'] = pd.to_datetime(df_label['@timestamp'])

In [25]:
df_label['time_stamp']

0        2020-05-02 03:20:47.897000+00:00
1        2020-05-02 03:20:47.897000+00:00
2        2020-05-02 03:19:36.692000+00:00
3        2020-05-02 03:19:36.692000+00:00
4        2020-05-02 03:19:37.733000+00:00
                       ...               
201043                                NaT
201044                                NaT
201045                                NaT
201046                                NaT
201047                                NaT
Name: time_stamp, Length: 201048, dtype: datetime64[ns, UTC]

In [30]:
# Looks to be what we want. start and end type per label per system.
df_label.sort_values('time_stamp').groupby(['TP_Binary_x', 'Hostname'])['time_stamp'].agg(['first','last', 'count'])

first  \
TP_Binary_x Hostname                                                  
False       NASHUA.dmevals.local   2020-05-02 02:55:26.494000+00:00   
            NEWYORK.dmevals.local  2020-05-02 02:55:27.499000+00:00   
            SCRANTON.dmevals.local 2020-05-02 02:55:28.511000+00:00   
            UTICA.dmevals.local    2020-05-02 02:55:26.493000+00:00   
True        NASHUA.dmevals.local   2020-05-02 03:09:29.929000+00:00   
            NEWYORK.dmevals.local  2020-05-02 03:04:05.749000+00:00   
            SCRANTON.dmevals.local 2020-05-02 02:55:57.730000+00:00   

                                                               last   count  
TP_Binary_x Hostname                                                         
False       NASHUA.dmevals.local   2020-05-02 03:28:20.170000+00:00   29021  
            NEWYORK.dmevals.local  2020-05-02 03:28:15.147000+00:00   23934  
            SCRANTON.dmevals.local 2020-05-02 03:28:13.119000+00:00  130428  
            UTICA.dmevals.local    2020-05-02 03:28:20.170000+00:00   11971  
True        NASHUA.dmevals.local   2020-05-02 03:17:53.817000+00:00      44  
            NEWYORK.dmevals.local  2020-05-02 03:04:05.749000+00:00       1  
            SCRANTON.dmevals.local 2020-05-02 03:19:53.189000+00:00    3535

In [35]:
# Check TP_Binary label for events within the 'malicious' time window
df_label.loc[(df_label['time_stamp'] >= '2020-05-02 03:09:29.929000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:17:53.817000+00:00') &
             (df_label['Hostname'] == 'NASHUA.dmevals.local')
            ]['TP_Binary_x'].value_counts()

False    11021
True        44
Name: TP_Binary_x, dtype: int64

In [38]:
# Create new column for time window labelling - start<>end time and hostname
df_label['TW_Binary'] = False
# Label malicious: start<>end time and hostname
df_label['TW_Binary'].loc[(df_label['time_stamp'] >= '2020-05-02 03:09:29.929000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:17:53.817000+00:00') &
             (df_label['Hostname'] == 'NASHUA.dmevals.local')
            ] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
# Looks like it worked. 
df_label['TW_Binary'].value_counts()

False    189983
True      11065
Name: TW_Binary, dtype: int64

In [41]:
# Implement Label for other hostname-time windows
df_label['TW_Binary'].loc[(df_label['time_stamp'] >= '2020-05-02 03:04:05.749000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:04:05.749000+00:00') &
             (df_label['Hostname'] == 'NEWYORK.dmevals.local')
            ] = True

In [42]:
df_label['TW_Binary'].value_counts()

False    189978
True      11070
Name: TW_Binary, dtype: int64

In [43]:
df_label['TW_Binary'].loc[(df_label['time_stamp'] >= '2020-05-02 02:55:57.730000+00:00') & 
             (df_label['time_stamp'] <= '2020-05-02 03:19:53.189000+00:00') &
             (df_label['Hostname'] == 'SCRANTON.dmevals.local')
            ] = True

In [50]:
df_label['TW_Binary'].value_counts()

False    111310
True      89738
Name: TW_Binary, dtype: int64

In [51]:
df_label['TP_Binary_x'].value_counts()

False    195354
True       3580
Name: TP_Binary_x, dtype: int64

In [52]:
df_label['TP_Binary_x'] = df_label['TP_Binary_x'].fillna(False)

In [53]:
df_label['TP_Binary_x'].value_counts()

False    197468
True       3580
Name: TP_Binary_x, dtype: int64

In [56]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
f1_score(df_label['TP_Binary_x'], df_label['TW_Binary'])

0.07672689084635333

In [57]:
roc_auc_score(df_label['TP_Binary_x'], df_label['TW_Binary'])

0.7818431340774201

In [58]:
accuracy_score(df_label['TP_Binary_x'], df_label['TW_Binary'])

0.5714555727985356

In [62]:
# Now Guid first degree label
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[(df_label['TP_Binary_x'] == True)].groupby(['ProcessGuid', 'TP_Label_x', 'TP_Binary_x'])['@timestamp', 'EventID'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


@timestamp  \
ProcessGuid                            TP_Label_x                                                            TP_Binary_x               
{47ab858c-cad9-5eac-0b00-000000000400} 0, 5.A.1. New Service                                                 True                  7   
{47ab858c-cc06-5eac-9402-000000000400} 0, 1.A.1. User Execution                                              True                  1   
{47ab858c-e13c-5eac-a903-000000000400} 0, 1.A.1. User Execution, 1.A.2. Masquerading                         True                  1   
                                       0, 1.A.3. Uncommonly Used Port                                        True                  1   
                                       0, 1.A.4. Standard Cryptographic Protocol                             True                  1   
                                       0, 3.A.1. Remote File Copy                                            True                  1   
{47ab858c-e144-5eac-ab03-000000000400} 0, 1.B.1. Command-Line Interface, 2.A.1. File and Directory Discovery True                  1   
{47ab858c-e14e-5eac-ac03-000000000400} 0, 1.B.2. PowerShell                                                  True                  1   
                                       0, 2.A.5. Data Staged, 4.A.1. Remote File Copy                        True                  1   
{47ab858c-e188-5eac-b003-000000000400} 0, 1.B.1. Command-Line Interface                                      True                  1   
{47ab858c-e18b-5eac-b103-000000000400} 0, 1.B.2. PowerShell                                                  True                  1   
                                       0, 3.B.1. Component Object Model Hijacking                            True                  1   
{47ab858c-e1e3-5eac-b603-000000000400} 0, 3.B.2. Bypass User Account Control                                 True                  1   
{47ab858c-e1e4-5eac-b803-000000000400} 0, 3.B.2. Bypass User Account Control                                 True                  1   
                                       0, 3.B.3. Commonly Used Port                                          True                 20   
{47ab858c-e1f8-5eac-bc03-000000000400} 0, 1.B.2. PowerShell                                                  True                  1   
                                       0, 3.C.1. Modify Registry                                             True                  1   
{47ab858c-e23d-5eac-c603-000000000400} 0, 4.A.2. PowerShell                                                  True                  1   
                                       0, 4.C.10. Execution through API, 4.C.12. Execution through API       True                  1   
                                       0, 5.B.1. Registry Run Keys / Startup Folder                          True                  1   
                                       0, 6.B.1. Private Keys                                                True                  1   
{47ab858c-e2ac-5eac-cb03-000000000400} 0, 4.B.2. File Deletion                                               True                  6   
{47ab858c-e2f2-5eac-d203-000000000400} 0, 4.B.3. File Deletion                                               True                  6   
{47ab858c-e305-5eac-d303-000000000400} 0, 4.B.4. File Deletion                                               True                  6   
{47ab858c-e342-5eac-d703-000000000400} 0, 6.A.3. Masquerading, 6.A.3. Masquerading                           True                  1   
{47ab858c-e374-5eac-d803-000000000400} 0, 4.A.2. PowerShell                                                  True                  1   
                                       0, 7.A.1. Screen Capture                                              True                  1   
                                       0, 7.B.2. Data Compressed                                             True                  1   
                                       0, 

In [63]:
# These guids look sufficiently 'malicious' and given the unique nature of the guid within the AD domain, should be a legitimate source for labelling.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_label.loc[(df_label['TP_Binary_x'] == True)].groupby(['ProcessGuid', 'TP_Label_x', 'CommandLine'])['@timestamp', 'EventID'].count())

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,,,@timestamp,EventID
ProcessGuid,TP_Label_x,CommandLine,,
{47ab858c-e13c-5eac-a903-000000000400},"0, 1.A.1. User Execution, 1.A.2. Masquerading","""C:\ProgramData\victim\â€®cod.3aka3.scr"" /S",1,1
{47ab858c-e144-5eac-ab03-000000000400},"0, 1.B.1. Command-Line Interface, 2.A.1. File and Directory Discovery","""C:\windows\system32\cmd.exe""",1,1
{47ab858c-e14e-5eac-ac03-000000000400},"0, 1.B.2. PowerShell",powershell,1,1
{47ab858c-e188-5eac-b003-000000000400},"0, 1.B.1. Command-Line Interface","""C:\windows\system32\cmd.exe""",1,1
{47ab858c-e18b-5eac-b103-000000000400},"0, 1.B.2. PowerShell",powershell,1,1
{47ab858c-e1e3-5eac-b603-000000000400},"0, 3.B.2. Bypass User Account Control","""C:\Windows\System32\control.exe"" /name Microsoft.BackupAndRestoreCenter",1,1
{47ab858c-e1e4-5eac-b803-000000000400},"0, 3.B.2. Bypass User Account Control","""PowerShell.exe"" -noni -noexit -ep bypass -window hidden -c ""sal a New-Object;Add-Type -AssemblyName 'System.Drawing'; $g=a System.Drawing.Bitmap('C:\Users\pbeesly\Downloads\monkey.png');$o=a Byte[] 4480;for($i=0; $i -le 6; $i++){foreach($x in(0..639)){$p=$g.GetPixel($x,$i);$o[$i*640+$x]=([math]::Floor(($p.B-band15)*16)-bor($p.G-band15))}};$g.Dispose();IEX([System.Text.Encoding]::ASCII.GetString($o[0..3932]))""",1,1
{47ab858c-e1f8-5eac-bc03-000000000400},"0, 1.B.2. PowerShell",powershell,1,1
{47ab858c-e23d-5eac-c603-000000000400},"0, 4.A.2. PowerShell",powershell.exe,1,1


In [72]:
bad_guids = df_label.loc[(df_label['TP_Binary_x'] == True) & (df_label.ProcessGuid.notnull())]['ProcessGuid'].unique()

In [74]:
df_label.loc[(df_label['ProcessGuid'].isin(bad_guids))].count()

@timestamp       22975
@version         22975
Category             0
Channel          22975
EventID          22975
                 ...  
TP_Binary_y        101
TP_Label_y         101
Source_Port_y      101
time_stamp       22975
TW_Binary        22975
Length: 548, dtype: int64

In [75]:
df_label['Guid1_Binary'] = False
df_label['Guid1_Binary'].loc[(df_label['ProcessGuid'].isin(bad_guids))] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [76]:
df_label['Guid1_Binary'].value_counts()

False    178073
True      22975
Name: Guid1_Binary, dtype: int64

In [ ]:
#from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
#f1_score(df_label['TP_Binary_x'], df_label['TW_Binary'])
#0.07672689084635333
#roc_auc_score(df_label['TP_Binary_x'], df_label['TW_Binary'])
#0.7818431340774201
#accuracy_score(df_label['TP_Binary_x'], df_label['TW_Binary'])
#0.5714555727985356

In [77]:
f1_score(df_label['TP_Binary_x'], df_label['Guid1_Binary'])

0.012050461306721898

In [128]:
from sklearn.metrics import classification_report
print(classification_report(df_label['TP_Binary_x'], df_label['Guid1_Binary']))

              precision    recall  f1-score   support

       False       0.98      0.88      0.93    197468
        True       0.01      0.04      0.01      3580

    accuracy                           0.87    201048
   macro avg       0.49      0.46      0.47    201048
weighted avg       0.96      0.87      0.91    201048



In [129]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(df_label['TP_Binary_x'], df_label['Guid1_Binary'])

-0.019358190592238866

In [131]:
cohen_kappa_score(df_label['TP_Binary_x'], df_label['TW_Binary'])

0.04398615697049946

In [143]:
from sklearn.metrics import cohen_kappa_score, accuracy_score, precision_score, recall_score, f1_score
label_vs_label = pd.DataFrame([['TP_Binary vs Time Window', 
                    cohen_kappa_score(df_label['TP_Binary_x'], df_label['TW_Binary']),
                    accuracy_score(df_label['TP_Binary_x'], df_label['TW_Binary']),
                    precision_score(df_label['TP_Binary_x'], df_label['TW_Binary']),
                    recall_score(df_label['TP_Binary_x'], df_label['TW_Binary']),
                    f1_score(df_label['TP_Binary_x'], df_label['TW_Binary'])
            ], ['TP_Binary vs Guid1',
                    cohen_kappa_score(df_label['TP_Binary_x'], df_label['Guid1_Binary']),
                    accuracy_score(df_label['TP_Binary_x'], df_label['Guid1_Binary']),
                    precision_score(df_label['TP_Binary_x'], df_label['Guid1_Binary']),
                    recall_score(df_label['TP_Binary_x'], df_label['Guid1_Binary']),
                    f1_score(df_label['TP_Binary_x'], df_label['Guid1_Binary'])
            ], ['Guid1 vs Time Window',
                    cohen_kappa_score(df_label['Guid1_Binary'], df_label['TW_Binary']),
                    accuracy_score(df_label['Guid1_Binary'], df_label['TW_Binary']),
                    precision_score(df_label['Guid1_Binary'], df_label['TW_Binary']),
                    recall_score(df_label['Guid1_Binary'], df_label['TW_Binary']),
                    f1_score(df_label['Guid1_Binary'], df_label['TW_Binary'])
               ]
            ], columns=['Experiment', 'Cohen_Kappa', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

In [144]:
# Doesn't really tell us much. Other than TP_Binary, Guid1_Binary, and TW_Binary are very different
# Suspicious that TP_Binary vs TW_Binary seems to be more 'in agreement'
# We must look underneath the single score metrics. 
label_vs_label

,Experiment,Cohen_Kappa,Accuracy,Precision,Recall,F1-Score
0,TP_Binary vs Time Window,0.043986,0.571456,0.039894,1.000000,0.076727
1,TP_Binary vs Guid1,-0.019358,0.869509,0.006964,0.044693,0.012050
2,Guid1 vs Time Window,0.273854,0.666980,0.254964,0.995865,0.405987


In [78]:
roc_auc_score(df_label['TP_Binary_x'], df_label['Guid1_Binary'])

0.4645775150273977

In [79]:
accuracy_score(df_label['TP_Binary_x'], df_label['Guid1_Binary'])

0.8695087740241136

In [80]:
# Interesting results. Now Guid vs Time window
f1_score(df_label['Guid1_Binary'], df_label['TW_Binary'])

0.4059868870494087

In [81]:
roc_auc_score(df_label['Guid1_Binary'], df_label['TW_Binary'])

0.8102061534874313

In [82]:
accuracy_score(df_label['Guid1_Binary'], df_label['TW_Binary'])

0.6669800246707254

In [83]:
# Now classful feature selection on those three labels to determine most 'relevant' AKA 'malicious' GUIDs

In [84]:
colName = 'ProcessGuid'

In [91]:
from sklearn.preprocessing import scale
from sklearn.linear_model.logistic import LogisticRegression

In [92]:
def get_classful_feature_importance(colName, df, label):
    x = pd.get_dummies(df[colName])
    # Random Forest Feature Importance
    rf = RandomForestClassifier(n_estimators = 100,
                               n_jobs = -1,
                               oob_score = True,
                               bootstrap = True,
                               random_state = 42,
                               max_depth = 30)
    rf.fit(x, df[label])

    indices = np.argsort(rf.feature_importances_)[::-1]
    feature_list = [x.columns[indices[f]] for f in range(x.shape[1])]  #names of features.
    ff = np.array(feature_list)

    # To get the importance according to each class:
    #def class_feature_importance(X, Y, feature_importances):
    N, M = x.shape
    x = scale(x)

    out = {}
    #for c in set(Y):
    out = dict(
        sorted(zip(range(M), np.mean(x[df[label]==True, :], axis=0)*rf.feature_importances_), key=lambda z: z[1])
    )

    pd_fs = pd.DataFrame(out.values(), ff[list(out.keys())], columns=['rfi'])

    # Logistic Regression Feature Importance
    #lr = LogisticRegression()
    #lr.fit(x, df[])

    #lri = pd.DataFrame(list(zip(lr.coef_[0], x.columns)),
    #                   columns=['coef', colName])

    #lri['coef_abs'] = lri.coef.abs()
    
    
    return pd_fs
    
    #result = pos_class_feature_importance(x, df.TP_Binary, rf.feature_importances_)
    #                                        df[label]
    

In [93]:
df_featureImp = get_classful_feature_importance('ProcessGuid', df_label, 'TP_Binary_x')

In [94]:
df_featureImp

,rfi
{47ab858c-daca-5eac-0203-000000000400},-0.054120
{47ab858c-e589-5eac-fa03-000000000400},-0.032291
{47ab858c-e6ad-5eac-0b00-000000000500},-0.008049
{5aa8ec29-e377-5eac-5f03-000000000400},-0.006429
{5aa8ec29-e463-5eac-6303-000000000400},-0.004213
...,...
{47ab858c-e2d4-5eac-cd03-000000000400},0.001465
{47ab858c-e558-5eac-f503-000000000400},0.002076
{47ab858c-e205-5eac-bf03-000000000400},0.002237
{47ab858c-e6a9-5eac-0400-000000000500},0.002263


In [96]:
df_label.loc[df_label.ProcessGuid.notnull()].groupby(['ProcessGuid', 'TP_Binary_x']).count()

,,@timestamp,@version,Category,Channel,EventID,EventReceivedTime,EventTime,EventType,ExecutionProcessID,Hostname,IpAddress,IpPort,Keywords,LogonGuid,Message,Opcode,OpcodeValue,ProviderGuid,RecordNumber,ServiceName,ServiceSid,Severity,SeverityValue,SourceModuleName,SourceModuleType,SourceName,Status,TargetDomainName,TargetUserName,Task,ThreadID,TicketEncryptionType,TicketOptions,TransmittedServices,Version,host_x,port,tags_x,TP_Label_x,Action,ActivityID,CalloutKey,CalloutName,ChangeType,Conditions,FilterId,FilterKey,FilterName,FilterType,LayerId,LayerKey,LayerName,ProcessId,ProviderKey,ProviderName,UserName,UserSid,Weight,AccessList,AccessMask,AccessReason,HandleId,ObjectName,ObjectServer,ObjectType,PrivilegeList,ProcessName,ResourceAttributes,RestrictedSidCount,SubjectDomainName,SubjectLogonId,SubjectUserName,SubjectUserSid,TransactionId,ShareLocalPath,ShareName,RelativeTargetName,CommandLine,MandatoryLabel,NewProcessId,NewProcessName,ParentProcessName,TargetLogonId,TargetUserSid,TokenElevationType,Service,DisabledPrivilegeList,EnabledPrivilegeList,CallerProcessId,CallerProcessName,TargetSid,AuthenticationPackageName,ElevatedToken,ImpersonationLevel,KeyLength,LmPackageName,LogonProcessName,LogonType,RestrictedAdminMode,TargetLinkedLogonId,TargetOutboundDomainName,TargetOutboundUserName,VirtualAccount,WorkstationName,SourceHandleId,SourceProcessId,TargetHandleId,TargetProcessId,AccountName,AccountType,ContextInfo,Domain,Payload,UserID,NewThreadId,RuleName,SourceImage,SourceProcessGuid,StartAddress,StartFunction,StartModule,TargetImage,TargetProcessGuid,UtcTime,MessageNumber,MessageTotal,Path,ScriptBlockId,ScriptBlockText,AlgorithmName,KeyName,KeyType,Operation,ReturnCode,Archived,Hashes,Image,IsExecutable,TargetFilename,User,CreationUtcTime,Company,Description,FileVersion,ImageLoaded,OriginalFileName,Product,Signature,SignatureStatus,Signed,DestinationHostname,DestinationIp,DestinationIsIpv6,DestinationPort,DestinationPortName,Initiated,Protocol,SourceHostname,SourceIp,SourceIsIpv6,SourcePort,SourcePortName,NewSd,OldSd,PipeName,CurrentDirectory,IntegrityLevel,LogonId,ParentCommandLine,ParentImage,ParentProcessGuid,ParentProcessId,TerminalSessionId,CallTrace,GrantedAccess,SourceProcessGUID,SourceThreadId,TargetProcessGUID,Device,TargetObject,Details,Application,FilterRTID,LayerRTID,SourceAddress,DestAddress,DestPort,Direction,RemoteMachineID,RemoteUserID,BootMenuPolicy,NewValue,NewValueType,ObjectValueName,OldValue,OldValueType,OperationType,ProfileUsed,RuleId,QueryName,QueryResults,QueryStatus,PreviousCreationUtcTime,EventCountTotal,EventIdx,GroupMembership,param1,Properties,Hash,PreAuthType,ClientProcessId,CountOfCredentialsReturned,ProcessCreationTime,ReadOperation,TargetName,Type,CalloutId,CalloutType,CorruptionActionState,DeviceName,DriveName,Config,IsTestConfig,DirtyPages,HiveName,HiveNameLength,KeysUpdated,PackageName,Workstation,DeviceNameLength,DeviceTime,DeviceVersionMajor,DeviceVersionMinor,FinalStatus,UserData,ServiceAccount,ServiceFileName,ServiceStartType,ServiceType,param10,param11,param2,param3,param4,param5,param6,param7,param8,param9,SubLayerKey,...,ProviderContextName,ProviderContextType,DwordVal,InterfaceGuid,InterfaceName,NewProfile,OldProfile,InstanceId,MaxRunspaces,MinRunspaces,CurrentStratumNumber,TimeSourceRefId,ReasonForRejection,Active,EdgeTraversal,EmbeddedContext,Flags,LocalAddresses,LocalOnlyMapped,LooseSourceMapped,ModifyingApplication,ModifyingUser,Origin,Profiles,RemoteAddresses,RuleStatus,SchemaVersion,SecurityOptions,CountNew,CountOld,EnabledNew,UpdateReason,NewTime,PreviousTime,NotificationPackageName,ShutdownActionType,ShutdownEventCode,ShutdownReason,BootMode,BuildVersion,MajorVersion,MinorVersion,QfeVersion,ServiceVersion,StartTime,EnableDisableReason,VsmPolicy,DCName,ProcessingMode,ProcessingTimeInMilliseconds,SupportInfo1,SupportInfo2,Group,IdleImplementation,IdleStateCount,MaximumPerformancePercent,MinimumPerformancePercent,MinimumThrottlePercent,NominalFrequency,Number,PerformanceImplem

In [107]:
df_label.loc[df_label.ProcessGuid.notnull()][['ProcessGuid', 'TP_Binary_x']].drop_duplicates().groupby(['ProcessGuid', 'TP_Binary_x']).count()

,
ProcessGuid,TP_Binary_x
{00000000-0000-0000-0000-000000000000},False
{32aa854b-c643-5eac-0100-000000000300},False
{32aa854b-c643-5eac-0300-000000000300},False
{32aa854b-c64c-5eac-0c00-000000000300},False
{32aa854b-c653-5eac-0f00-000000000300},False
...,...
{6bbf237a-e79a-5eac-8402-000000000400},False
{6bbf237a-e7a2-5eac-8502-000000000400},False
{6bbf237a-e7a2-5eac-8602-000000000400},False


In [108]:
# bad_guids = df_label.loc[(df_label['TP_Binary_x'] == True) & (df_label.ProcessGuid.notnull())]['ProcessGuid'].unique()
bad_guids

array(['{47ab858c-e13c-5eac-a903-000000000400}',
       '{47ab858c-e2f2-5eac-d203-000000000400}',
       '{47ab858c-e342-5eac-d703-000000000400}',
       '{47ab858c-e23d-5eac-c603-000000000400}',
       '{5aa8ec29-e5b8-5eac-7903-000000000400}',
       '{47ab858c-e374-5eac-d803-000000000400}',
       '{47ab858c-e2ac-5eac-cb03-000000000400}',
       '{47ab858c-e305-5eac-d303-000000000400}',
       '{47ab858c-e18b-5eac-b103-000000000400}',
       '{5aa8ec29-e618-5eac-7e03-000000000400}',
       '{47ab858c-e442-5eac-ed03-000000000400}',
       '{47ab858c-e1f8-5eac-bc03-000000000400}',
       '{5aa8ec29-e5e4-5eac-7a03-000000000400}',
       '{47ab858c-cad9-5eac-0b00-000000000400}',
       '{47ab858c-e1e4-5eac-b803-000000000400}',
       '{5aa8ec29-cad1-5eac-0100-000000000400}',
       '{5aa8ec29-e52e-5eac-6b03-000000000400}',
       '{47ab858c-e49f-5eac-f103-000000000400}',
       '{5aa8ec29-e56d-5eac-6e03-000000000400}',
       '{5aa8ec29-e63e-5eac-8203-000000000400}',
       '{47ab858c-cc

In [115]:
pd_bad_guids = pd.DataFrame(index=bad_guids)

In [116]:
pd_bad_guids['Label'] = True

In [165]:
pd_all_guids = df_label.loc[df_label.ProcessGuid.notnull()][['ProcessGuid', 'TP_Binary_x']]

In [168]:
pd_all_guids.drop_duplicates()

,ProcessGuid,TP_Binary_x
640,{47ab858c-e374-5eac-d803-000000000400},False
643,{47ab858c-e710-5eac-c300-000000000500},False
644,{5aa8ec29-dadd-5eac-be02-000000000400},False
645,{5aa8ec29-e215-5eac-5403-000000000400},False
646,{47ab858c-e23d-5eac-c603-000000000400},False
...,...,...
194208,{5aa8ec29-dadd-5eac-bd02-000000000400},False
194209,{47ab858c-db23-5eac-2c03-000000000400},False
197713,{47ab858c-cae0-5eac-4000-000000000400},False
198053,{5aa8ec29-e4ec-5eac-6803-000000000400},True


In [195]:
bad_guids = df_label.loc[(df_label['TP_Binary_x'] == True) & (df_label.ProcessGuid.notnull())]['ProcessGuid'].unique()
pd_all_guids = df_label.loc[df_label.ProcessGuid.notnull()][['ProcessGuid', 'TP_Binary_x']]
pd_all_guids['true_label'] = False
pd_all_guids['true_label'].loc[pd_all_guids.ProcessGuid.isin(bad_guids)] = True

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [196]:
pd_all_guids = pd_all_guids.drop(columns=['TP_Binary_x']).drop_duplicates()
pd_all_guids

,ProcessGuid,true_label
640,{47ab858c-e374-5eac-d803-000000000400},True
643,{47ab858c-e710-5eac-c300-000000000500},False
644,{5aa8ec29-dadd-5eac-be02-000000000400},False
645,{5aa8ec29-e215-5eac-5403-000000000400},False
646,{47ab858c-e23d-5eac-c603-000000000400},True
...,...,...
191388,{47ab858c-dad3-5eac-0e03-000000000400},False
194208,{5aa8ec29-dadd-5eac-bd02-000000000400},False
194209,{47ab858c-db23-5eac-2c03-000000000400},False
197713,{47ab858c-cae0-5eac-4000-000000000400},False


In [201]:
df_featureImp = df_featureImp.reset_index()

In [208]:
df_featureImp.columns = ['ProcessGuid', 'rfi']

In [209]:
df_featureImp

,ProcessGuid,rfi
0,{47ab858c-daca-5eac-0203-000000000400},-0.054120
1,{47ab858c-e589-5eac-fa03-000000000400},-0.032291
2,{47ab858c-e6ad-5eac-0b00-000000000500},-0.008049
3,{5aa8ec29-e377-5eac-5f03-000000000400},-0.006429
4,{5aa8ec29-e463-5eac-6303-000000000400},-0.004213
...,...,...
645,{47ab858c-e2d4-5eac-cd03-000000000400},0.001465
646,{47ab858c-e558-5eac-f503-000000000400},0.002076
647,{47ab858c-e205-5eac-bf03-000000000400},0.002237
648,{47ab858c-e6a9-5eac-0400-000000000500},0.002263


In [210]:
pd.merge(pd_all_guids, df_featureImp, on='ProcessGuid')

,ProcessGuid,true_label,rfi
0,{47ab858c-e374-5eac-d803-000000000400},True,-0.000000
1,{47ab858c-e710-5eac-c300-000000000500},False,-0.000019
2,{5aa8ec29-dadd-5eac-be02-000000000400},False,-0.000000
3,{5aa8ec29-e215-5eac-5403-000000000400},False,-0.000000
4,{47ab858c-e23d-5eac-c603-000000000400},True,-0.000000
...,...,...,...
645,{47ab858c-dad3-5eac-0e03-000000000400},False,-0.000000
646,{5aa8ec29-dadd-5eac-bd02-000000000400},False,-0.000000
647,{47ab858c-db23-5eac-2c03-000000000400},False,-0.000000
648,{47ab858c-cae0-5eac-4000-000000000400},False,-0.000000


In [275]:
def get_classful_feature_importance(colName, df, label):
    
    bad_features = df.loc[(df['TP_Binary_x'] == True) & (df[colName].notnull())]['ProcessGuid'].unique()
    pd_all_feature = df.loc[df[colName].notnull()][[colName, 'TP_Binary_x']]
    pd_all_feature['true_label'] = False
    pd_all_feature['true_label'].loc[pd_all_feature[colName].isin(bad_features)] = True
    pd_all_feature = pd_all_feature.drop(columns=['TP_Binary_x']).drop_duplicates()

    #return pd_all_feature

    x = pd.get_dummies(df_label[colName])
    y = df_label[label]
    X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=42)

    # Random Forest Feature Importance

    # Random Forest Feature Importance
    # Change max_depth 30->40 to attempt 100% accuracy in training set.
    #     -> no change to accuracy performance.
    rf = RandomForestClassifier(n_estimators = 100,
                               n_jobs = -1,
                               oob_score = True,
                               bootstrap = True,
                               random_state = 42,
                               max_depth = 40)
    rf.fit(X_train, y_train)
    
    print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
    print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))
    
    print("RF train f1 score: %0.3f" % f1_score(y_train, (rf.predict(X_train))))
    print("RF test f1 score: %0.3f" % f1_score(y_test, (rf.predict(X_test))))
    
    indices = np.argsort(rf.feature_importances_)[::-1]
    feature_list = [x.columns[indices[f]] for f in range(x.shape[1])]  #names of features.
    ff = np.array(feature_list)

    # To get the importance according to each class:
    #def class_feature_importance(X, Y, feature_importances):
    N, M = x.shape
    x_scale = scale(x)

    out = {}
    #for c in set(Y):
    out = dict(
        sorted(zip(range(M), np.mean(x_scale[df[label]==True, :], axis=0)*rf.feature_importances_), key=lambda z: z[1])
    )

    pd_fs = pd.DataFrame(out.values(), ff[list(out.keys())], columns=['rfi'])

    pd_fs = pd_fs.reset_index()
    pd_fs.columns = [colName, 'rfi']
    
    pd_fs_out = pd.merge(pd_all_feature, pd_fs, on=colName)
    
    # Logistic Regression Feature Importance
    #x = pd.get_dummies(df_label[colName])
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    
    print("LR train accuracy: %0.3f" % lr.score(X_train, y_train))
    print("LR test accuracy: %0.3f" % lr.score(X_test, y_test))
    
    print("LR train f1 score: %0.3f" % f1_score(y_train, (lr.predict(X_train))))
    print("LR test f1 score: %0.3f" % f1_score(y_test, (lr.predict(X_test))))

    lri = pd.DataFrame(list(zip(lr.coef_[0], x.columns)),
                       columns=['coef', colName])

    lri['coef_abs'] = lri.coef.abs()
    
    pd_fs_out = pd.merge(pd_fs_out, lri, on=colName)

    
    return pd_fs_out
    
    #result = pos_class_feature_importance(x, df.TP_Binary, rf.feature_importances_)
    #                                        df[label]
    

In [276]:
classful_feature_importance = get_classful_feature_importance('ProcessGuid', df_label, 'TP_Binary_x')

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


RF train accuracy: 0.982
RF test accuracy: 0.982
RF train f1 score: 0.000
RF test f1 score: 0.000
LR train accuracy: 0.982
LR test accuracy: 0.982
LR train f1 score: 0.000
LR test f1 score: 0.000


In [277]:
classful_feature_importance

,ProcessGuid,true_label,rfi,coef,coef_abs
0,{47ab858c-e374-5eac-d803-000000000400},True,-0.000000,-1.840172,1.840172
1,{47ab858c-e710-5eac-c300-000000000500},False,0.000045,-0.733955,0.733955
2,{5aa8ec29-dadd-5eac-be02-000000000400},False,-0.000000,-0.344138,0.344138
3,{5aa8ec29-e215-5eac-5403-000000000400},False,-0.000000,-2.001600,2.001600
4,{47ab858c-e23d-5eac-c603-000000000400},True,-0.000000,-1.882868,1.882868
...,...,...,...,...,...
645,{47ab858c-dad3-5eac-0e03-000000000400},False,-0.000000,0.000000,0.000000
646,{5aa8ec29-dadd-5eac-bd02-000000000400},False,-0.000000,-0.028694,0.028694
647,{47ab858c-db23-5eac-2c03-000000000400},False,-0.000010,-0.028694,0.028694
648,{47ab858c-cae0-5eac-4000-000000000400},False,-0.000361,-0.028694,0.028694


In [270]:
classful_feature_importance.loc[classful_feature_importance.true_label == True].sort_values(by=['coef'], ascending=False)

,ProcessGuid,true_label,rfi,coef,coef_abs
482,{47ab858c-e305-5eac-d303-000000000400},True,-0.000000,1.329857,1.329857
314,{47ab858c-e2f2-5eac-d203-000000000400},True,-0.000000,1.201898,1.201898
459,{47ab858c-e2ac-5eac-cb03-000000000400},True,-0.000000,1.201898,1.201898
359,{47ab858c-cad9-5eac-0b00-000000000400},True,-0.000000,1.194095,1.194095
539,{47ab858c-cc06-5eac-9402-000000000400},True,-0.000002,0.704249,0.704249
521,{5aa8ec29-e52e-5eac-6b03-000000000400},True,-0.000000,0.580969,0.580969
143,{47ab858c-e6b2-5eac-4d00-000000000500},True,-0.000022,0.580969,0.580969
501,{5aa8ec29-e4ec-5eac-6803-000000000400},True,0.002391,0.446588,0.446588
442,{47ab858c-e342-5eac-d703-000000000400},True,-0.000000,0.446588,0.446588
12,{47ab858c-e1e4-5eac-b803-000000000400},True,-0.000000,0.442152,0.442152


In [272]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(classful_feature_importance.sort_values(by=['coef'], ascending=False).head(30))

,ProcessGuid,true_label,rfi,coef,coef_abs
482,{47ab858c-e305-5eac-d303-000000000400},True,-0.000000e+00,1.329857,1.329857
314,{47ab858c-e2f2-5eac-d203-000000000400},True,-0.000000e+00,1.201898,1.201898
459,{47ab858c-e2ac-5eac-cb03-000000000400},True,-0.000000e+00,1.201898,1.201898
359,{47ab858c-cad9-5eac-0b00-000000000400},True,-0.000000e+00,1.194095,1.194095
539,{47ab858c-cc06-5eac-9402-000000000400},True,-1.970279e-06,0.704249,0.704249
143,{47ab858c-e6b2-5eac-4d00-000000000500},True,-2.235760e-05,0.580969,0.580969
521,{5aa8ec29-e52e-5eac-6b03-000000000400},True,-0.000000e+00,0.580969,0.580969
501,{5aa8ec29-e4ec-5eac-6803-000000000400},True,2.390768e-03,0.446588,0.446588
442,{47ab858c-e342-5eac-d703-000000000400},True,-0.000000e+00,0.446588,0.446588
12,{47ab858c-e1e4-5eac-b803-000000000400},True,-0.000000e+00,0.442152,0.442152


In [269]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(classful_feature_importance.sort_values(by=['coef_abs'], ascending=False).head(40))

,ProcessGuid,true_label,rfi,coef,coef_abs
218,{47ab858c-e6c0-5eac-7600-000000000500},False,-0.000000e+00,-4.303601,4.303601
205,{32aa854b-e288-5eac-5203-000000000300},False,-0.000000e+00,-3.287483,3.287483
38,{32aa854b-e60c-5eac-6103-000000000300},False,-2.738422e-04,-3.205325,3.205325
201,{47ab858c-cada-5eac-1000-000000000400},False,-0.000000e+00,-2.940134,2.940134
202,{5aa8ec29-cad8-5eac-1000-000000000400},False,-3.367388e-05,-2.825791,2.825791
203,{47ab858c-dac2-5eac-e802-000000000400},False,-0.000000e+00,-2.666982,2.666982
216,{47ab858c-cae1-5eac-4600-000000000400},False,-8.016574e-05,-2.584602,2.584602
350,{5aa8ec29-dad4-5eac-a702-000000000400},False,-0.000000e+00,-2.576079,2.576079
196,{47ab858c-e69e-5eac-0100-000000000500},False,-2.370544e-06,-2.551994,2.551994
164,{47ab858c-e710-5eac-bf00-000000000500},False,-0.000000e+00,-2.457675,2.457675


In [240]:
from sklearn.model_selection import train_test_split
colName = 'ProcessGuid'
label = 'TP_Binary_x'

x = pd.get_dummies(df_label[colName])
y = df_label[label]
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=42)

# Random Forest Feature Importance
# Change max_depth 30->40 to attempt 100% accuracy in training set.
#     -> no change to accuracy performance.
rf = RandomForestClassifier(n_estimators = 100,
                           n_jobs = -1,
                           oob_score = True,
                           bootstrap = True,
                           random_state = 42,
                           max_depth = 40)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=40, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [241]:
print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))

RF train accuracy: 0.982
RF test accuracy: 0.982


In [ ]:
>>> y_true = [0, 1, 0, 0, 1, 0]
>>> y_pred = [0, 1, 0, 0, 0, 1]
>>> balanced_accuracy_score(y_true, y_pred)
0.625

In [282]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_train, rf.predict(X_train), adjusted=True)

0.0

In [281]:
f1_score(y_train, rf.predict(X_train))

0.0

In [286]:
classful_feature_importance = get_classful_feature_importance('ProcessGuid', df_label, 'Guid1_Binary')

/home/jake/miniconda3/envs/py3msc/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


RF train accuracy: 1.000
RF test accuracy: 1.000
RF train f1 score: 0.999
RF test f1 score: 0.999
LR train accuracy: 1.000
LR test accuracy: 1.000
LR train f1 score: 0.999
LR test f1 score: 0.999


In [288]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(classful_feature_importance.sort_values(by=['coef'], ascending=False).head(40))

,ProcessGuid,true_label,rfi,coef,coef_abs
0,{47ab858c-e374-5eac-d803-000000000400},True,-0.000000e+00,12.812824,12.812824
4,{47ab858c-e23d-5eac-c603-000000000400},True,-0.000000e+00,10.731460,10.731460
215,{5aa8ec29-e5e4-5eac-7a03-000000000400},True,-0.000000e+00,10.401326,10.401326
197,{47ab858c-e14e-5eac-ac03-000000000400},True,-5.189092e-06,10.386531,10.386531
200,{47ab858c-e1f8-5eac-bc03-000000000400},True,-0.000000e+00,9.679722,9.679722
9,{47ab858c-e18b-5eac-b103-000000000400},True,-0.000000e+00,9.620464,9.620464
190,{5aa8ec29-e5b8-5eac-7903-000000000400},True,-0.000000e+00,9.454103,9.454103
12,{47ab858c-e1e4-5eac-b803-000000000400},True,-5.541146e-07,9.451147,9.451147
192,{5aa8ec29-cad1-5eac-0100-000000000400},True,-0.000000e+00,8.061537,8.061537
7,{47ab858c-e13c-5eac-a903-000000000400},True,-3.721838e-06,7.899279,7.899279


In [244]:
sorted_idx = rf.feature_importances_.argsort()

In [245]:
tree_feature_importances = rf.feature_importances_

In [246]:
tree_feature_importances

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.36208664e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.46785749e-04,
       0.00000000e+00, 0.00000000e+00, 1.93355359e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.54230142e-05, 0.00000000e+00,
       8.16503759e-05, 9.06568102e-05, 0.00000000e+00, 0.00000000e+00,
       7.73682487e-04, 1.69785215e-05, 0.00000000e+00, 1.20576670e-03,
       0.00000000e+00, 0.00000000e+00, 1.34433133e-04, 6.75500816e-04,
       5.39446857e-02, 0.00000000e+00, 0.00000000e+00, 2.17821927e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.85226812e-02,
       0.00000000e+00, 0.00000000e+00, 4.70255468e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.26588346e-02, 7.69351002e-05,
       0.00000000e+00, 3.43356826e-02, 1.07617721e-02, 0.00000000e+00,
      

In [247]:
# Logistic Regression Feature Importance
lr = LogisticRegression()
lr.fit(X_train, y_train)

lri = pd.DataFrame(list(zip(lr.coef_[0], x.columns)),
                   columns=['coef', colName])

lri['coef_abs'] = lri.coef.abs()

In [248]:
lri

,coef,ProcessGuid,coef_abs
0,-0.344138,{00000000-0000-0000-0000-000000000000},0.344138
1,-0.767006,{32aa854b-c643-5eac-0100-000000000300},0.767006
2,-0.028694,{32aa854b-c643-5eac-0300-000000000300},0.028694
3,-1.877982,{32aa854b-c64c-5eac-0c00-000000000300},1.877982
4,-0.487106,{32aa854b-c653-5eac-0f00-000000000300},0.487106
...,...,...,...
645,-0.461158,{6bbf237a-e79a-5eac-8402-000000000400},0.461158
646,-0.237742,{6bbf237a-e7a2-5eac-8502-000000000400},0.237742
647,-0.390805,{6bbf237a-e7a2-5eac-8602-000000000400},0.390805
648,-0.344138,{6bbf237a-e853-5eac-8702-000000000400},0.344138


In [285]:
balanced_accuracy_score(y_train, lr.predict(X_train))

0.5

In [284]:
f1_score(y_train, lr.predict(X_train))

0.0

In [191]:
pd.concat([df_featureImp, pd_all_guids])

,rfi,ProcessGuid,true_label
{47ab858c-daca-5eac-0203-000000000400},-0.054120,NaN,NaN
{47ab858c-e589-5eac-fa03-000000000400},-0.032291,NaN,NaN
{47ab858c-e6ad-5eac-0b00-000000000500},-0.008049,NaN,NaN
{5aa8ec29-e377-5eac-5f03-000000000400},-0.006429,NaN,NaN
{5aa8ec29-e463-5eac-6303-000000000400},-0.004213,NaN,NaN
...,...,...,...
191388,NaN,{47ab858c-dad3-5eac-0e03-000000000400},False
194208,NaN,{5aa8ec29-dadd-5eac-bd02-000000000400},False
194209,NaN,{47ab858c-db23-5eac-2c03-000000000400},False
197713,NaN,{47ab858c-cae0-5eac-4000-000000000400},False


In [149]:
pd_closer = pd.concat([pd_all_guids, pd_bad_guids.reindex(pd_all_guids.index)], axis=1)

In [164]:
pd.DataFrame(pd_closer['Label'])

,Label
ProcessGuid,
{47ab858c-e374-5eac-d803-000000000400},True
{47ab858c-e710-5eac-c300-000000000500},NaN
{5aa8ec29-dadd-5eac-be02-000000000400},NaN
{5aa8ec29-e215-5eac-5403-000000000400},NaN
{47ab858c-e23d-5eac-c603-000000000400},True
...,...
{5aa8ec29-dadd-5eac-bd02-000000000400},NaN
{47ab858c-db23-5eac-2c03-000000000400},NaN
{47ab858c-cae0-5eac-4000-000000000400},NaN
